## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
## set gpu usage
import tensorflow as tf
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True, per_process_gpu_memory_fraction = 0.9))
session = tf.Session(config=config)
keras.backend.tensorflow_backend.set_session(session)


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

# Xception 架構
model=keras.applications.xception.Xception(include_top=False, weights='imagenet', input_tensor=input_tensor,pooling=None, classes=2)
    
# Resnet 50 架構
# model=keras.applications.ResNet50(input_shape=(32,32,3), include_top=False, weights='imagenet',pooling=None, classes=10)
model.summary()


Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 15, 15, 32)    864         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, 15, 15, 32)    128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, 15, 15, 32)    0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

## 添加層數

In [2]:
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(output_dim=128, activation='relu')(x)
x=Dropout(p=0.1)(x)
predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))

Model深度： 136


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  """


## 鎖定特定幾層不要更新權重

In [3]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


## Training

In [5]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
50000/50000 [==============================] - 53s - loss: 1.3946 - acc: 0.5124    
Epoch 2/100
50000/50000 [==============================] - 51s - loss: 1.0264 - acc: 0.6477    
Epoch 3/100
50000/50000 [==============================] - 52s - loss: 0.9087 - acc: 0.6865    
Epoch 4/100
50000/50000 [==============================] - 51s - loss: 0.8229 - acc: 0.7161    
Epoch 5/100
50000/50000 [==============================] - 51s - loss: 0.7484 - acc: 0.7419    
Epoch 6/100
50000/50000 [==============================] - 51s - loss: 0.6713 - acc: 0.7685    
Epoch 7/100
50000/50000 [==============================] - 52s - loss: 0.6055 - acc: 0.7910    
Epoch 8/100
50000/50000 [==============================] - 51s - loss: 0.5526 - acc: 0.8130    
Epoch 9/100
50000/50000 [==============================] - 51s - loss: 0.4997 - acc: 0.8289    
Epoch 10/100
50000/50000 [==============================] - ETA: 0s - loss: 0.4446 - acc: 0.8489- - 52s - loss: 0.4447 - acc: 0.8489    

KeyboardInterrupt: 